### Nedostajuće vrednosti

Neretko u skupovima podataka kojima raspolažemo postoje nedostajuće vrednosti. U `Python` bibliotekama nedostajućim vrednostima se smatraju `None` i `NaN` vrednosti. Ovaj skup vrednosti se može proširiti i svim vrednostima koje nisu iz zadovoljavajućih opsega ili kategorija za posmatrani problem. Na primer, redni broj meseca ne može imati vrednost 13 ili prosečan broj pacijenata jedne ordinacije ne može biti negativan. U tom smislu bi se ove vrednosti mogle zameniti nedostajućim. 

Cilj narednih primera je da približe rad sa funkcijama koje omogućavaju manipulaciju nedostajućim vrednostima, kao i da uvedu primene linearne regresije u oceni nedostajućih vrednosti. 

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import metrics

Prvo ćemo generisati proizvoljni skup tačaka u ravni.

In [3]:
np.random.seed(7)

In [4]:
N = 10
x = np.random.randn(N)
y = np.random.rand(N)

In [5]:
points = pd.DataFrame({'x':x, 'y': y})

Brz uvid u brojnost atributa i njihove vrednosti možemo dobiti pozivom funkcije `info`.

In [6]:
points.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x       10 non-null     float64
 1   y       10 non-null     float64
dtypes: float64(2)
memory usage: 288.0 bytes


Dalje ćemo zbog demonstracije funkcija neke vrednosti proglasiti nedostajućima.

In [7]:
points.loc[5, 'y'] = None
points.loc[6, 'y'] = np.nan
points.loc[9,'x'] = np.nan
points.loc[9,'y'] = np.nan

In [8]:
points

,x,y
0,1.690526,0.380941
1,-0.465937,0.065936
2,0.032820,0.288146
3,0.407516,0.909594
4,-0.788923,0.213385
5,0.002066,NaN
6,-0.000890,NaN
7,-1.754724,0.024899
8,1.017658,0.600549
9,NaN,NaN


Sada možemo videti da nam funkcija `info` ukazuje na prisustvo nedostajućih vrednosti.

In [9]:
points.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x       9 non-null      float64
 1   y       7 non-null      float64
dtypes: float64(2)
memory usage: 288.0 bytes


Funkcija `isna` Pandas biblioteke proverava koje vrednosti su nedostajuće. Ravnopravno se može koristiti i funkcija `isnull`.

In [10]:
points.isna()

,x,y
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
5,False,True
6,False,True
7,False,False
8,False,False
9,True,True


Funkcija `notna` Pandas biblioteke proverava koje vrednosti nisu nedostajuće. Ravnopravno se može koristiti i funkcija `notnull`.

In [11]:
points.notna()

,x,y
0,True,True
1,True,True
2,True,True
3,True,True
4,True,True
5,True,False
6,True,False
7,True,True
8,True,True
9,False,False


Funkcija `isnan` numpy biblioteke proverava da li je vrednost nedostajuća. 

In [12]:
np.isnan(points.loc[5, 'x'])

False

In [13]:
np.isnan(points.loc[5, 'y'])

True

Dalje su pobrojane neke tehnike koje imamo na raspolaganju ukoliko su nedostajuće vrednosti prisutne u skupu podataka sa kojim radimo.


#### Brisanje nedostajućih vrednosti 

Za brisanje nedostajućih vrednosti može se koristi funkcija `dropna` Pandas biblioteke. Njenim parametrima se utiče na brisanje vrsta ili kolona (parametar `axis`), način brisanja (parametar `how`) i rezultujuću vrednost poziva funkcije (parametar `inplace`). 

Na primer, sledeći poziv će obrisati devetu tačku jer je naglašeno da treba obrisati vrste u kojima su sve vrednosti nedostajuće. 

In [14]:
points.dropna(axis = 0, how = 'all', inplace = True)

In [15]:
points

,x,y
0,1.690526,0.380941
1,-0.465937,0.065936
2,0.032820,0.288146
3,0.407516,0.909594
4,-0.788923,0.213385
5,0.002066,NaN
6,-0.000890,NaN
7,-1.754724,0.024899
8,1.017658,0.600549


Sledeći poziv bi obrisao petu i šestu tačku jer je naglašeno da treba obrisati vrste u kojima je barem jedna vrednost nedostajuća.

In [16]:
#points.dropna(axis = 0, how = 'any', inplace = True)

In [17]:
#points

#### Zamena nedostajuće vrednosti prosečnom vrednošću, medijanom ili modom

Sledećim blokom koda se prvo izračunava srednja vrednost svojstva `y`, a potom se korišćenjem funkcije `replace` vrši zamena nedostajućih vrednosti. Parametar `to_replace` ukazuje na vrednosti koje treba zameniti, a parametar `value` na vrednost koju treba koristiti prilikom zamene.

In [18]:
y_mean = points[points['y'].notna()]['y'].mean()
y_mean

0.3547785865252053

In [19]:
points['y'].replace(to_replace=np.nan, value=y_mean, inplace=True)

In [20]:
points

,x,y
0,1.690526,0.380941
1,-0.465937,0.065936
2,0.032820,0.288146
3,0.407516,0.909594
4,-0.788923,0.213385
5,0.002066,0.354779
6,-0.000890,0.354779
7,-1.754724,0.024899
8,1.017658,0.600549


#### Zamena nedostajućih vrednosti nekim proizvoljnim vrednostima iz skupa vrednosti

Funkcija `random_imputation` zamenjuje nedostajuće vrednosti u koloni određenoj parametrom `feature` na slučajan način.

In [21]:
def random_imputation(data, feature):
    number_of_missing_values = data[feature].isna().sum()
    observed_values = data.loc[data[feature].notna(), feature]
    
    new_values = np.random.choice(observed_values, number_of_missing_values)
    
    data.loc[data[feature].isna(), feature] = new_values
    
    return data

In [22]:
random_imputation(points, 'y')

,x,y
0,1.690526,0.380941
1,-0.465937,0.065936
2,0.032820,0.288146
3,0.407516,0.909594
4,-0.788923,0.213385
5,0.002066,0.354779
6,-0.000890,0.354779
7,-1.754724,0.024899
8,1.017658,0.600549


#### Aproksimiranje nedostajućih vrednosti na osnovu vrednosti sused

U primeru koji sledi hteli bismo da na osnovu poznatih vrednosti atributa korišćenjem linearne regresije aproksimiramo vrednosti atributa koje nedostaju. Skup podataka sa kojim ćemo raditi je preuzet sa [adrese](https://www.kaggle.com/ranjeetjain3/seaborn-tips-dataset) i sadrži informacije o napojnicama zaposlenih u jednom restoranu. Osim visine napojnice poznati su i iznos ukupnog računa, informacija o kom obroku se radilo, danu u nedelji, veličini grupe, polu osobe koja je platila i sekciji restorana za pušače ili nepušače. 

Prvo ćemo učitati podatke i pripremiti ih za dalji rad.

In [23]:
data = pd.read_csv('data/tips.csv')

In [24]:
data.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  244 non-null    float64
 1   tip         244 non-null    float64
 2   sex         244 non-null    object 
 3   smoker      244 non-null    object 
 4   day         244 non-null    object 
 5   time        244 non-null    object 
 6   size        244 non-null    int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 13.5+ KB


In [26]:
N = data.shape[0]
N

244

Prvo ćemo pripremiti skup podataka tako što ćemo sve kategoričke atribute zameniti odgovarajućim numeričkim ili kombinacijom numeričkih. 

Za zamenu kategoričkih atributa možemo koristiti funkciju `replace` biblioteke Pandas. Potrebno je da prvo izvidimo koje vrednosti atribut može imati, a potom i da izdvojenim vrednostima pridružimo odgovarajuće numeričke kodove.

In [27]:
data['smoker'].unique()

array(['No', 'Yes'], dtype=object)

In [28]:
data['smoker'].replace({'No': 0, 'Yes': 1}, inplace=True)

In [29]:
data['sex'].unique()

array(['Female', 'Male'], dtype=object)

In [30]:
data['sex'].replace({'Female': 0, 'Male': 1}, inplace=True)

Drugi način pripreme ovakvih atributa demonstriraćemo kroz korišćenje funkcije `get_dummies` biblioteke Pandas. Ovaj način je podesniji ukoliko atribut uzima više vrednosti. 

Funkcijom `get_dummies` se vrši takozvano `one-hot` kodiranje kojim se odgovarajućoj vrednosti pridružuje vektor svih nula i jedne jedinice čija pozicija u vektoru odgovara poziciji te vrednosti u skupu vrednosti. Na primer, ako su vrednosti u skupu *Sun*, *Sat*, *Thur* i *Fri*, njima pridruženi vektori su (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0) i (0, 0, 0, 1). 

Ukoliko se raspolaže skupom od `k` vrednosti, obično se na ovaj način kodira `k-1` vrednosti jer se vektor nula indirektno veže za nerazmatranu vrednost. Ovaj pristup je dobar zbog kontrolisane redudantnosti i korelisanosti dobijenih atributa. U funkciji `get_dummies` to dozvoljava parametar `drop_first`.

Na ovaj način pripremićemo kategoričke atribute `day` i `time`.

In [31]:
data['day'].unique()

array(['Sun', 'Sat', 'Thur', 'Fri'], dtype=object)

In [32]:
day = pd.get_dummies(data['day'], prefix = 'day', drop_first=True)

In [33]:
day.head()

,day_Sat,day_Sun,day_Thur
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0


In [34]:
data['time'].unique()

array(['Dinner', 'Lunch'], dtype=object)

In [35]:
time = pd.get_dummies(data['time'], prefix = 'time', drop_first=True)

In [36]:
time.head()

,time_Lunch
0,0
1,0
2,0
3,0
4,0


Kolone vrednosti koje smo dobili u prethodnim koracima prvo ćemo nadovezati na polazni skup podataka korišćenjem funkcije `concat`.

In [37]:
data = pd.concat([data, day, time], axis = 1)

In [38]:
data.head()

,total_bill,tip,sex,smoker,day,time,size,day_Sat,day_Sun,day_Thur,time_Lunch
0,16.99,1.01,0,0,Sun,Dinner,2,0,1,0,0
1,10.34,1.66,1,0,Sun,Dinner,3,0,1,0,0
2,21.01,3.50,1,0,Sun,Dinner,3,0,1,0,0
3,23.68,3.31,1,0,Sun,Dinner,2,0,1,0,0
4,24.59,3.61,0,0,Sun,Dinner,4,0,1,0,0


Potom ćemo obrisati originalne kolone `day` i `time` jer nam više neće biti potrebne. Za brisanje ćemo iskoristiti funkciju `drop`.

In [39]:
data.drop(['day','time'], axis = 1, inplace= True)

In [40]:
data.head()

,total_bill,tip,sex,smoker,size,day_Sat,day_Sun,day_Thur,time_Lunch
0,16.99,1.01,0,0,2,0,1,0,0
1,10.34,1.66,1,0,3,0,1,0,0
2,21.01,3.50,1,0,3,0,1,0,0
3,23.68,3.31,1,0,2,0,1,0,0
4,24.59,3.61,0,0,4,0,1,0,0


Na kraju, pripremićemo i atribut `size` koji je numeričkog tipa ali sa diskretnim skupom vrednosti.

In [41]:
data['size'].unique()

array([2, 3, 4, 1, 6, 5])

In [42]:
size = pd.get_dummies(data['size'], prefix = 'size', drop_first=True)

In [43]:
size.head()

,size_2,size_3,size_4,size_5,size_6
0,1,0,0,0,0
1,0,1,0,0,0
2,0,1,0,0,0
3,1,0,0,0,0
4,0,0,1,0,0


In [44]:
data = pd.concat([data, size], axis = 1)

In [45]:
data.drop(['size'], axis = 1, inplace= True)

In [46]:
data.head()

,total_bill,tip,sex,smoker,day_Sat,day_Sun,day_Thur,time_Lunch,size_2,size_3,size_4,size_5,size_6
0,16.99,1.01,0,0,0,1,0,0,1,0,0,0,0
1,10.34,1.66,1,0,0,1,0,0,0,1,0,0,0
2,21.01,3.50,1,0,0,1,0,0,0,1,0,0,0
3,23.68,3.31,1,0,0,1,0,0,1,0,0,0,0
4,24.59,3.61,0,0,0,1,0,0,0,0,1,0,0


Neke vrednosti napojnica ćemo veštački proglasiti nedostajućim. Vrednosti ćemo pre brisanja sačuvati kao deo test skupa zbog evaluacije našeg modela. Za predikciju ćemo koristiti vrednosti atributa `total_bill`, ekvivalente `size` atributa i `smoker` atribut. 

In [47]:
mask_indexes = np.random.randint(0, N, 50)

Izdvojićemo programski (više demonstracije radi :)) kolone koje ćemo koristiti za razvoj modela.

In [48]:
model_columns = [column for column in data.columns if column.startswith('size')]

In [49]:
model_columns

['size_2', 'size_3', 'size_4', 'size_5', 'size_6']

In [50]:
model_columns.append('smoker')
model_columns.append('total_bill')

In [51]:
model_columns

['size_2', 'size_3', 'size_4', 'size_5', 'size_6', 'smoker', 'total_bill']

Kreiraćemo dalje skup za testiranje, a potom i skup za treniranje. Izdvojićemo vrednosti poznatih atributa i na osnovu njih naučiti da predviđamo vrednost atributa koji ima nedostajuće vrednosti.

In [52]:
test = data.loc[mask_indexes]
y_test = test['tip']
X_test = test[model_columns]

In [53]:
train = data.drop(mask_indexes, axis = 0)
y_train = train['tip']
X_train = train[model_columns]

Dalje možemo izvršiti pripremu skupova. Ovde treba imati na umu da naš skup kombinuje kategoričke i kontinualne vrednosti. Prilikom pripreme treba standardizovati samo kontinualne kako bi ekvivalenti kategoričkih zadržali svoj smisao. Kod koji sledi simulira opštu proceduru. 

In [54]:
scaler = preprocessing.StandardScaler()

In [55]:
scaler.fit(X_train[['total_bill']])

StandardScaler()

In [56]:
X_train_scaled = np.zeros(X_train.shape)

In [57]:
X_train_scaled[:, 0:6] = X_train[['size_2', 'size_3', 'size_4', 'size_5', 'size_6', 'smoker']]

In [58]:
X_train_scaled[:, 6] = scaler.transform(X_train[['total_bill']]).flatten()

In [59]:
X_train.shape

(200, 7)

In [60]:
X_test_scaled = np.zeros(X_test.shape)

In [61]:
X_test_scaled[:, 0:6] = X_test[['size_2', 'size_3', 'size_4', 'size_5', 'size_6', 'smoker']]

In [62]:
X_test_scaled[:, 6] = scaler.transform(X_test[['total_bill']]).flatten()

In [63]:
X_test.shape

(50, 7)

Model kojim ćemo predviđati nedostajuće vrednosti će biti jednostavan linearni model, ali se u praksi mogu koristiti i kompleksniji modeli. 

In [64]:
model = linear_model.LinearRegression()
model.fit(X_train_scaled, y_train)

LinearRegression()

Može ispitati i ocenu našeg modela.

In [65]:
y_predicted = model.predict(X_test_scaled)

In [66]:
metrics.r2_score(y_test, y_predicted)

0.3872867854568124

Ovaj pristup aproksimacije nedostajućih vrednosti se često koristi u sistemima preporuka za nove proizvode u rešavanju takozvanog hladnog starta (eng. cold start) gde zbog nepoznavanja mišljenja ili ocena korisnika proizvod ne može da se probije na listi preporuka.